In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
! cat /proc/cpuinfo
! nvidia-smi

import cv2

print(cv2.__version__) # Should print 4.4.0
print(cv2.cuda)

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs mmio_stale_data retbleed
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtua

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.180639571000029
GPU (s):
0.08915402899992841
GPU speedup over CPU: 24x


In [ ]:
# %cd /content

In [ ]:
# !git clone https://github.com/opencv/opencv.git
# !git clone https://github.com/opencv/opencv_contrib.git

In [ ]:
# !mkdir /content/build
# %cd /content/build

In [ ]:
# !cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules -DBUILD_SHARED_LIBS=OFF -DBUILD_TESTS=OFF -DBUILD_PERF_TESTS=OFF -DBUILD_EXAMPLES=OFF -DWITH_OPENEXR=OFF -DWITH_CUDA=ON -DWITH_CUBLAS=ON -DWITH_CUDNN=ON -DOPENCV_DNN_CUDA=ON /content/opencv

In [ ]:
# !make -j8 install

In [ ]:
# cv2 install is NOT persistant, next colab allocation will wipe it all clean, so you need to copy the so to some folder on your own drive
# !mkdir  "/content/drive/My Drive/cv2_cuda"
# !cp  /content/build/lib/python3/cv2.cpython-310-x86_64-linux-gnu.so   "/content/drive/My Drive/cv2_cuda"

# Next time you want to use it, copy it back to your current workdir
!cp "/content/drive/My Drive/cv2_cuda/cv2.cpython-36m-x86_64-linux-gnu.so" .

cp: cannot stat '/content/drive/My Drive/cv2_cuda/cv2.cpython-36m-x86_64-linux-gnu.so': No such file or directory


In [ ]:
import numpy as np
import pandas as pd
import pathlib
import imageio
import os
import cv2

print(cv2.__version__) # Should print 4.4.0
print(cv2.cuda)

4.7.0
<module 'cv2.cuda' (<_frozen_importlib_external._NamespaceLoader object at 0x7f7e1ec0dfc0>)>


In [ ]:
%cd /content/
print(os.getcwd())

/content
/content


In [ ]:
base_dir = "./drive/MyDrive/kium_medical_composition/"

train_img_dir = base_dir + "train_set/images/"
# test_img_dir = base_dir + "test_set/images/"

train_imgs = [train_img_dir + "{}".format(i) for i in os.listdir(train_img_dir) ]
# test_imgs = [test_img_dir + "{}".format(i) for i in os.listdir(test_img_dir) ]

print("Total train images: ", len(train_imgs))
# print("Total test images: ", len(test_imgs))

Total train images:  9016


In [ ]:
tf.image.resize(train_imgs, [244,244])

In [ ]:
import cv2

# Preprocessing Step
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
X = [] # images
y = []

count=0

img_size = 224

used_patient_num = 50

for patient_idx in range(0, used_patient_num):

    try :
        img_elem = []
        img_elem_equalize = []

        for sample_idx in range(0, 8):
            image_index = (patient_idx * 8) + sample_idx
            img_elem.append(cv2.imread(train_imgs[image_index],cv2.IMREAD_GRAYSCALE))

        # resize
        for i in range(0, 8):
            img_elem_equalize.append(cv2.equalizeHist(img_elem[i]))
            img_elem_equalize[i] = cv2.resize(img_elem_equalize[i],(img_size,img_size),interpolation=cv2.INTER_CUBIC)

            img_elem[i] = cv2.resize(img_elem[i],(img_size,img_size),interpolation=cv2.INTER_CUBIC)

        # convert image to 2D to 3D
        img = np.dstack([img_elem[0], img_elem[1], img_elem[2], img_elem[3], img_elem[4], img_elem[5], img_elem[6], img_elem[7]])
        img_eq = np.dstack([img_elem_equalize[0], img_elem_equalize[1], img_elem_equalize[2], img_elem_equalize[3], img_elem_equalize[4], img_elem_equalize[5], img_elem_equalize[6], img_elem_equalize[7]])

        # convert greyscale image to RGB
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Normalalize Image
        img = img.astype(np.float32)/255.
        img_eq = img_eq.astype(np.float32)/255.

        print(img.shape)
        print(img_eq.shape)

        count=count+1

        X.append(img_eq)

    except:
        continue

# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import matplotlib.pyplot as plt

# plt.figure()
# plt.subplot(1, 2, 1)

# print(np.array(X[0]).shape)
# plt.imshow(img_eq)
# plt.imshow(np.array(X[0]))

# print(img)
# print("--------------------------------------------------------")
# print(img_eq)

In [ ]:
# Get the labels
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
train_label = pd.read_csv(base_dir + "train_set/train.csv")

aneurysm = train_label.Aneurysm
y = aneurysm[0:used_patient_num]

# location = train_label.drop(columns=['Index', 'Aneurysm']) # Not used currently
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
X = np.array(X)
y = np.array(y)

# Change the shape of datasets to be fit the CNN Model
# X = X.reshape((used_patient_num, 224, 224, 8, 1))

In [ ]:
X.shape
print(y)

[0 1 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 1 0 0 1
 1 1 0 1 1 0 0 1 0 1 1 1 0]


In [ ]:
# Modeling Learning Step 1 (MS)
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

x_train, y_train = X, y

x_train, x_val, y_train, y_val = train_test_split( x_train, y_train, test_size=0.2,random_state=111 )

def cnn_block(input_layer,
                    cnn_num=3,
                    channel=64,
                    block_num=1):

    x = input_layer

    for c_num in range(cnn_num):
      x = keras.layers.Conv2D(filters=channel,
                              kernel_size=(3,3),
                              activation='relu',
                              kernel_initializer='he_normal',
                              padding='same',
                              name=f'block{block_num}_conv{c_num}')(x)

    x = tf.keras.layers.MaxPooling2D(
          pool_size=(2,2),
          strides=2,
          name=f'block{block_num}_pooling')(x)

    return x

def build_vgg(input_shape=(224,224,8),
              cnn_num_list=[2,2,3,3,3],
              channel_list=[64,128,256,512,512],
              num_classes=10):

    input = keras.layers.Input(shape=input_shape)
    x = input

    for i , (cnn_num, channel) in enumerate(zip(cnn_num_list, channel_list)):
        x = cnn_block(x, cnn_num, channel, i)

    x = keras.layers.Flatten(name='flatten')(x)
    x = keras.layers.Dense(4096, activation='relu', name='FC-4096-1')(x)
    x = keras.layers.Dense(4096, activation='relu', name='FC-4096-2')(x)
    x = keras.layers.Dense(1000, activation='relu', name='FC-1000')(x)
    x = keras.layers.Dense(1, activation='sigmoid', name='predictions')(x)


    model = keras.Model(
        inputs = input,
        outputs = x
    )

    return model

vgg_16 = build_vgg()

vgg_16.compile(
    loss = keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics = ['accuracy'],
)

# history_16 = vgg_16.fit(x_train, y_train, epochs=10)

history_16 = vgg_16.fit(x_train, y_train,
                        batch_size=128,
                        epochs=10,
                        validation_data=(x_val, y_val))
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Epoch 1/10
1/1 [==============================] - 15s 15s/step - loss: 0.0000e+00 - accuracy: 0.6250 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 2/10
1/1 [==============================] - 1s 598ms/step - loss: 0.0000e+00 - accuracy: 0.3750 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 3/10
1/1 [==============================] - 1s 598ms/step - loss: 0.0000e+00 - accuracy: 0.3750 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 4/10
1/1 [==============================] - 1s 596ms/step - loss: 0.0000e+00 - accuracy: 0.3750 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 5/10
1/1 [==============================] - 1s 595ms/step - loss: 0.0000e+00 - accuracy: 0.3750 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 6/10
1/1 [==============================] - 1s 591ms/step - loss: 0.0000e+00 - accuracy: 0.3750 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 7/10
1/1 [==============================] - 1s 594ms/step - loss: 0.0000e+00 - accuracy: 0.3750 - val_loss: